In [ ]:
import os
import argparse
from pathlib import Path
import warnings
import torch
from torchvision import transforms, datasets
from pathlib import Path
import torch.nn as nn
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms
from autoattack1 import AutoAttack
from resnet import *

In [ ]:
#In the first step we trained a model with standard (model), robust (model_r) and pure robust dataset (model_rr)

In [ ]:

# Step 1: Load CIFAR-10 Dataset
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Step 2: Define Data Loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)
# Step 3: Initialize ResNet Model
model = ResNet50()

# Step 4: Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# Step 5: Training Loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # Step 6: Validation/Test Loop
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = correct / total
    print(f'Epoch {epoch + 1}/{num_epochs}, Accuracy: {100 * accuracy:.2f}%')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
torch.save(model.state_dict(), 'resnet50_cifar10.pt')

print('Trained model saved!')

Do the same for robust and pure robust model

In [ ]:
data_dir = './data'
norm = 'L2'
epsilon = 0.5
model_path = './resnet50_cifar10.pt'
n_ex = 10000
individual = False
save_dir = './results'
batch_size = 500
log_path = './log_file.txt'
version = 'standard'
state_path = None

In [ ]:
#Perturbing images assuming the adversary has complete knowledge of the model

In [ ]:
# Load your model
model = ResNet50()
ckpt = torch.load(model_path)
model.load_state_dict(ckpt)
model.cuda()
model.eval()

# Load data
transform_list = [transforms.ToTensor()]
transform_chain = transforms.Compose(transform_list)
item = datasets.CIFAR10(root=data_dir, train=False, transform=transform_chain, download=True)
test_loader = torch.utils.data.DataLoader(item, batch_size=1000, shuffle=False, num_workers=0)

# Create save directory
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Load attack
adversary = AutoAttack(model, norm=norm, eps=epsilon, log_path=log_path, version=version)

l = [x for (x, y) in test_loader]
x_test = torch.cat(l, 0)
l = [y for (x, y) in test_loader]
y_test = torch.cat(l, 0)

# Example of custom version
if version == 'custom':
    adversary.attacks_to_run = ['apgd-ce', 'fab']
    adversary.apgd.n_restarts = 2
    adversary.fab.n_restarts = 2

# Run attack and save images
with torch.no_grad():
    if not individual:
        adv_complete = adversary.run_standard_evaluation(x_test[:n_ex], y_test[:n_ex], bs=batch_size, state_path=state_path)

        torch.save({'adv_complete': adv_complete}, f'{save_dir}/aa_{version}_1_{n_ex}_eps_{epsilon:.5f}-nr.pth')

    else:
        # Individual version, each attack is run on all test points
        adv_complete = adversary.run_standard_evaluation_individual(x_test[:n_ex], y_test[:n_ex], bs=batch_size)

        torch.save(adv_complete, f'{save_dir}/aa_{version}_individual_1_{n_ex}_eps_{epsilon:.5f}-nr.pth')

In [ ]:
# Load saved data
saved_data_path = f'{save_dir}/aa_{version}_1_{n_ex}_eps_{epsilon:.5f}-nr.pth'
loaded_data = torch.load(saved_data_path)

# Access the loaded data
adv_complete_loaded_nr = loaded_data['adv_complete']

In [ ]:
#now give the perturbed images with robust auto attack to standard model

In [ ]:
import torch
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

model = ResNet50()
model.load_state_dict(torch.load('resnet50_cifar10.pt'))
model.eval()

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

cifar10_test = CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(cifar10_test, batch_size=1, shuffle=False)

adv_labels = cifar10_test.targets[:10000]

# Evaluate on adversarial examples
num_correct = 0

with torch.no_grad():
    for i in range(1000):
        random_index = torch.randint(len(test_loader.dataset), (1,)).item()
        input_image, target = adv_complete_loaded_nr[random_index], adv_labels[random_index]
        input_image = input_image.unsqueeze(0)

        # Forward pass
        output = model(input_image)
        _, predicted = torch.max(output.data, 1)

        # Check accuracy
        if predicted.item() == target:
            num_correct += 1

# Calculate accuracy
accuracy = num_correct / 1000
print("Accuracy on Adversarial Examples: {:.2%}".format(accuracy))

Files already downloaded and verified
Accuracy on Adversarial Examples: 36.00%


repeat the same experiment for standard, robust and purified robust dataset, models ans attacks